In [ ]:
# import data
import pandas as pd
import cv2

data = []

category = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

# get text data
with open ("/kaggle/input/garbage-classification/zero-indexed-files.txt", "r") as text:
    data = text.read().splitlines()
    

df = pd.DataFrame(columns=['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash'])

for img_pair in data[:10]:
    print(img_pair)
    path = f"/kaggle/input/garbage-classification/Garbage classification/Garbage classification/{category[img_pair[len(img_pair) - 1]]}/{img_pair[:-2]}"
    print(path)
    img = cv2.imread(path)
    print(img)
    
    


In [ ]:
!pip install flickrapi

In [ ]:
import flickrapi
import xml.etree.ElementTree as ET
import urllib.request
import os

In [ ]:
# open the .txt list of bird species and read them in as a list 
with open("/kaggle/input/recyclableitemslabels2/recyclable-items version 2.txt") as recyclable_items_file:
    recyclable_items = [line.strip() for line in recyclable_items_file]

In [ ]:
#get an api key and secret from flickr
api_key = u'150adfe3c2bb6b37e2160d88724730c6'
api_secret = u'ec7d59b70b6720a6'
flickr = flickrapi.FlickrAPI(api_key, api_secret)

In [ ]:
from tqdm import tqdm
for item in recyclable_items:
    photos = flickr.walk(text=item,
                        tag_mode='all',
                        tags=item,
                        extras='url_c',
                        privacy_filters = 1,
                        per_page=500,         
                        sort='relevance')
    for i, photo in tqdm(enumerate(photos)):
        url = photo.get('url_c')
        #if an error occurs just keep moving
        try:
            # first 10 images are for test
            if i <= 19:
                TestDirectory = f'/kaggle/working/ITEMS/test/{item}'
                #check if directory exists
                if not os.path.exists(TestDirectory):
                    os.makedirs(TestDirectory)
                #create file path
                filepath = os.path.join(TestDirectory, f'{i+1}.jpg')
                # Download image from the url and save it to '00001.jpg'
                urllib.request.urlretrieve(url, filepath)

            #next five images are for validation after model has been created
            elif i < 25 and i > 19:
                #create a validation folder in another location
                ValidateDirectory = f'/kaggle/working/Item_Validate/validate/{item}'
                #check if directory exists
                if not os.path.exists(ValidateDirectory):
                    os.makedirs(ValidateDirectory)
                #create file path
                filepath = os.path.join(ValidateDirectory, f'{i+1}.jpg')
                # Download image from the url and save it to '00001.jpg'
                urllib.request.urlretrieve(url, filepath)

            #stop after 200 total images
            elif i > 500:
                break

            #all remaining images go to training
            else:
                TrainDirectory = f'/kaggle/working/ITEMS/train/{item}'
                #check if directory exists
                if not os.path.exists(TrainDirectory):
                    os.makedirs(TrainDirectory)
                #create file path    
                filepath = os.path.join(TrainDirectory, f'{i-9}.jpg')
                # Download image from the url and save it to '00001.jpg'
                urllib.request.urlretrieve(url, filepath)
        except:
            pass
import shutil
shutil.make_archive("items", 'zip', "/kaggle/working/ITEMS")

In [ ]:
import shutil
shutil.make_archive("validation", 'zip', "/kaggle/working/Item_Validate/validate/")

In [ ]:
import shutil
shutil.rmtree('/kaggle/working/')

# Creating Dataset

In [ ]:
KaggleDatasets().get_gcs_path()

In [ ]:
import tensorflow as tf


BATCH_SIZE = 1
IMG_SIZE = (256, 256)

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/ecycle-images/E Cycle Image Data/recyclable items/train",
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/ecycle-images/E Cycle Image Data/recyclable items/test",
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)



In [ ]:
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_dataset).numpy())
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset).numpy())

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(28, kernel_size=(3, 3), input_shape=(256, 256, 3)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())  # Flattening the 2D arrays for fully connected layers
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(28, activation=tf.nn.softmax))

model.summary()

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""
: Model Summary
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10 ,validation_data=(test_dataset))

# Testing from TensorFlow MNIST
MARK: https://www.tensorflow.org/datasets/keras_example

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
# ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(8)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
from tqdm import tqdm

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

tqdm(model.fit(
    ds_train,
    epochs=2,
    validation_data=ds_test,
))

# Testing Code from TDS

MARK: https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428

In [ ]:
EPOCHS = 32
iter = train_dataset.makeOneShotIterator()
x, y = iter.get_next()
# make a simple model
net = tf.layers.dense(x, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)
loss = tf.losses.mean_squared_error(prediction, y) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

In [ ]:
print(train_dataset.

In [ ]:
import tensorflow_datasets as tfds
ds_numpy = tfds.as_numpy(train_dataset)